In [113]:
import pandas as pd
import numpy as np

### 1. 데이터 정제하기 : 데이터프레임명['컬럼명'] = np.where(조건식, yes값, no값)

In [114]:
selloutData = pd.read_csv("./dataset/kopo_channel_seasonality_new.csv")
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


#### 1-1. QTY가 마이너스 값일때 "NEW_QTY" 0입력, 아닌것은 "QTY"값 입력 

In [115]:
selloutData["NEW_QTY"] = np.where(selloutData["QTY"] < 0, 0, selloutData["QTY"])

In [116]:
selloutData.dtypes  ##데이터 타입 확인

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

#### 1-2. 53주차 제거하기

In [117]:
##selloutData에서 53주차인 값 삭제하기(!=는 '아니다'라는 뜻 : 53주차가 아닌것을 반환하라는 뜻!)
refinedData = selloutData[selloutData.YEARWEEK % 100 != 53]
refinedData[refinedData.YEARWEEK % 100 > 52] ##삭제되었는지 확인

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY


### 2. 연주차 정보 추출하기

In [118]:
##WEEK이라는 컬럼명을 새로 만들어주는데 거기에 들어가는 데이터값은 YEARWEEK의 WEKK부분(100으로 나눈 나머지 값)
refinedData["WEEK"] = selloutData["YEARWEEK"]%100  

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [119]:
### YEAR를 만들어줄 함수 만들기
def customFunction(inputColumn):
#     inputColumn = 201602
    yearValue = inputColumn/100
    yearValue = int(yearValue)
    return yearValue

In [120]:
##YEAR라는 컬럼명을 만들어주고 거기에 들어가는 값은 YEARWEEK에 있는 YEAR부분(함수를 활용하여 YEAR추출)
refinedData["YEAR"] = refinedData["YEARWEEK"].apply(customFunction)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### 3. 데이터 정렬하기 : 데이터프레임명.sort_values(['컬럼1', '컬럼2',...]

In [123]:
# sortKey 생성
sortKey = ["REGIONID", "PRODUCT", "YEARWEEK"]  

In [124]:
sortedData = refinedData.sort_values(sortKey)

In [125]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
298,A00,PRODUCT34,201401,661.0,661.0,1,2014
1757,A00,PRODUCT34,201402,679.0,679.0,2,2014
3125,A00,PRODUCT34,201403,578.0,578.0,3,2014
205,A00,PRODUCT34,201404,532.0,532.0,4,2014
4369,A00,PRODUCT34,201405,516.0,516.0,5,2014


In [126]:
##정렬한 데이터들의 인덱스를 다시 리셋하고 싶을때(drop=True)
step1Data = sortedData.reset_index(drop=True)  
##(drop=True)를 사용하지 않으면 정렬한 데이터는 그대로 유지되면서 앞에 인덱스번호만 순서대로 매겨짐

### 4. 롤링함수 사용(표준편차 구하기) 
: 데이터프레임명.롤링할컬럼명.rolling(window= , center=True).mean()

In [127]:
### 롤링함수를 사용하기 전에는 정렬을 반드시 사용해야 함!!
## min_periods 사용방법 인지하기~!

In [128]:
step1Data["MA"] = step1Data.NEW_QTY.rolling(window=15, center=True, min_periods=1).mean()

In [129]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000


In [57]:
# step1Data.to_csv("./result.csv")   ##데이터 저장하기

In [58]:
pwd  ##현재 위치 확인하기

'C:\\Users\\HP\\Python_Fintech'

### 5. 그룹바이 : 데이터프레임명.groupby(그룹키)[["컬럼명"]].agg(["mean","std"])["QTY_NEW"]

In [168]:
# filteredData = step1Data[(step1Data.REGIONID=="A00") &
#                         (step1Data.PRODUCT=="PRODUCT34")]

In [169]:
# 그룹키 정하기
groupKey = ["REGIONID", "PRODUCT", "YEAR"]

In [170]:
groupData = step1Data.groupby(groupKey)[["NEW_QTY"]].mean() #평균을 구해야하기때문에 mean으로 설정

In [171]:
groupData = groupData.reset_index()

In [172]:
groupData.rename(columns={"NEW_QTY":"GROUP"}, inplace = True)

### 6. merge : 
pd.merge(데이터프레임명1, 데이터프레임명2, left_on="데이터프레임1 조인 키" "right_on='데이터프레임2 조인 키', how= "left")   

In [173]:
mergeData = pd.merge(step1Data, groupData, how="left", on=["REGIONID", "PRODUCT", "YEAR"])
# mergeData = pd.merge(step1Data, groupData, how="left" on=["REGIONID", "PRODUCT", "YEAR"]) 
# 으로 변경 가능 : 두개의 데이터들의 컬럼명이 같기때문

In [174]:
mergeData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,GROUP
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000,275.961538


### 8. group by한 데이터를 롤링하기

In [175]:
groupKey = ["REGIONID", "PRODUCT"]

In [176]:
# groupDataSet = mergeData.groupby(groupKey)
# groupDataSet

In [177]:
# oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[1]) #그룹데이타의 1개의 그룹 확인 단계
# oneGroup.head()

In [178]:
# len(list(groupDataSet.groups)) #데이타의 그룹 길이 확인

In [184]:
## 그룹바이함수 만들기
def groupRolling(oneGroup): #oneGroup은 임의로 지정한 변수
    indexGroupData = oneGroup.reset_index(drop=True) ##인덱스그룹데이터에 원그룹의 인덱스를 리셋시켜라
    indexGroupData["MA2"] = indexGroupData["MA"].rolling(window=5, center=True, min_periods=1).mean() 
        ## => 인덱스그룹데이타에 "MA2"라는 컬럼을 넣을건데, 그 컬럼에는 인덱스그룹데이타의 
        ## "MA"의 롤링한 값(본인이 센터가 되어 위아래로 날개 2개씩 총 5개 데이터)의 평균을 넣어라
        ## indexGroupData는 내가 함수를 적용시킬 변수를 담을 임의의 변수
    
    return indexGroupData ##함수에는 꼭 return값을 넣어줘야함! => indexGroupData를 리턴값으로 !

In [185]:
finalResult = mergeData.groupby(groupKey).apply(groupRolling)

In [188]:
finalResult.to_csv("./finalResult.csv")

### 9. 컬럼명이 같은 데이터를 위아래로 조인 : pd.concat([데이터프레임명1, 데이터프레임명2])

In [191]:
fullData = pd.read_csv("./dataset/kopo_channel_seasonality_new.csv")

In [193]:
a01Data = fullData[fullData.REGIONID=="A01"]  ##fullData의 REGIONID중 "A01"데이타를 a01Data에 삽입
len(a01Data) ##a01Data의 길이 확인

3611

In [194]:
a20Data = fullData[fullData.REGIONID=="A20"]
len(a20Data)

471

In [198]:
resultConcat = pd.concat([a01Data, a20Data], axis=0)  
    ##axis=0으로 하면 축(아래)로 붙이는것, 1이면 옆으로 붙이는것

In [197]:
len(resultConcat)

4082